# DICOM Crawling Code
Given a list of folders, this script will resave all data from DICOMs stored in those folders. The pixel data will be stored in *pixel_data.h5* and the metadata from the DICOM headers will be stored in *metadata.csv*. The HDF5 and CSV files support efficient data reading and evaluation. Note that running this notebook is equivalent to calling *run_crawler.py* from the command line.

This script can be used to:
1. Save all DICOM metadata without resaving any pixel data (write_pixeldata = False)
2. Save all DICOM metadata + a 2d image from each DICOM (write_pixeldata = True, eval_3d_scans = False)
3. Save DICOM metadata + a 3d image from each series found in the provided DICOMs; this script will automatically group all DICOM files within a folder that originated from the same series (write_pixeldata = True, eval_3d_scans = True)

Each set of DICOMs stored by this script will be identified by a unique Scan ID, which will be used in all output files. The Scan ID will be automatically generated as "path_to_folder_where_DICOM_is_stored_UID", where the UID is the Series Instance UID (if 3d scans are being stored) or the SOP Instance UID (if 2d scans are being stored). If the UID does not exist in the DICOM metadata, the Scan ID will simply be "path_to_folder_where_DICOM_is_stored_noUID".

**Example**

/path/to/folder/1 contains the files: image1_slice1.dcm, image1_slice2.dcm, image1_slice3.dcm
<br>/path/to/folder/2 contains the files: image2_slice1.dcm, image2_slice2.dcm
<br>/path/to/folder/3 contains the files: image3_slice1.dcm, image3_slice2.dcm, image4_slice1.dcm, image4_slice2.dcm
<br>*Note: It is not required that the images and slices be numbered, as they are in the toy example above.*

By setting dicom_folders = ['/path/to/folder/1', '/path/to/folder/2', '/path/to/folder/3'], write_pixeldata = True, and eval_3d_scans = True, this code will generate the following three files:

1. pixel_data.h5: contains four datasets, each of which is an array of pixel values from image1, image2, image3, and image4. The name of the dataset is the Scan ID. Note that the function is capable of identifying different series within the same folder as long as the Series Instance UID is present in the DICOM metadata.

2. metadata.csv: contains one row each with all metadata from image1, image2, image3, and image4. Each row in metadata.csv corresponds to one dataset in pixel_data.h5, identified by their shared Scan ID.
<br>Note: If storing 3d scans, the metadata is stored for only one DICOM per scan; in other words, all DICOMs in the same folder with the same Series Instance UID are assumed to have the same relevant metadata when grouping DICOMs to store 3d scans. 

3. unreadable_files.csv: contains a row for each scan that did not successfully have metadata and/or pixel data extracted; if a Scan ID appears in unreadable_files.csv, it should not appear in either pixel_data.h5 or metadata.csv. A brief reason indicating why the read failed is provided in each row.


**Notes**
 - Sometimes, there will be duplicate tags in the DICOM header. If this is the case, all values associated with that tag will be stored in the same cell in metadata.csv. For example, if the tag "Example tag (0000, 0000)" appears twice in the DICOM header, once with the value "1" and once with the value "2", then the stored metadata for "Example tag (0000, 0000)" will appear as "DUPLICATE TAGS IN DICOM:1/2".

 - If the header row in metadata.csv contains two phrases connected by an underscore (e.g., "Referenced Image Sequence_Group Length (0008, 0000)") this indicates that the second tag ("Group Length") was nested inside of the first tag ("Referenced Image Sequence").

 - When grouping DICOMs to store 3d scans, this code will check all DICOMs in a given folder for unique Series Instance UIDs. If multiple Series Instance UIDs are present in the folder, indicating more than one scan present in that folder, then DICOMs with the same UID will be grouped and stored together. 

 - If output files pixel_data.h5 and metadata.csv already exist, running this code will append onto the existing files with new scans; any scans already stored will be ignored. This is useful if you acquire new data you want to add to your dataset, or if your dicom_crawl() call is stopped sometime during execution. 

### Start by importing required packages and functions

In [ ]:
from crawler_utils import dicom_crawl
import glob
import os

### Set the parameters

In [ ]:
# Define a list of directories, each of which should contain DICOMs. All DICOMs directly inside these directories will be processed.
# Note: If you provide a list that contains elements that are NOT directories, those elements will be ignored
# Note: If you provide a directory that contains files other than DICOMs, those non-DICOM files will be ignored
# Note: The function glob.glob('c:/all_dicom_folders/*') may be useful here
dicom_folders = ['/path/to/folder/1', '/path/to/folder/2']

# Define a directory to store outputs
storage_folder = os.getcwd()

# Define a unique identifier for output filenames; can be None
# ex: output_id = 'study1' results in outputs 'pixel_data_study1.h5' and 'metadata_study1.csv'
# ex: output_id = None results in outputs 'pixel_data.h5' and 'metadata.csv'
output_id = None

# Define number of processors; note that if saving 2d scans, parallelization will be ignored
n_procs = 1

# Turn on/off functionality to create h5 of all pixel data; writing pixel data increases run time significantly
# Set to True to write all pixel data, False to write no pixel data
write_pixeldata = False 

# Choose whether to evaluate 3d series or 2d images
# If eval_3d_scans = True, dicom_crawl() will find all scans with the same series instance UID and stack them in order into a 3d image, then save the 3d stack in the h5 and save one line of metadata/scan
# If eval_3d_scans = False, dicom_crawl() will save each individual DICOM file's pixel data as a 2d image in the h5 file and each DICOM file's metadata will be written to a CSV
eval_3d_scans = True

# Choose whether to parallelize over the folders by setting par_over_folder = True, or over the scans within a folder by setting par_over_folder = 0
# Note this parallelization is only used when evaluating 3d data.
# If you have many folders in dicom_folders, each with O(1) scan, set the parameter below to True
# If you have many scans per folder, set the parameter below to False to have the code parallelize over scans within a folder
par_over_folder = False

### Run DICOM crawling code

In [ ]:
# Crawl dicoms
dicom_crawl(dicom_folders, storage_folder, output_id, n_procs, write_pixeldata, eval_3d_scans, par_over_folder)
print("Finished running DICOM crawling code.")